In [8]:
# from https://stackoverflow.com/questions/42710879/write-two-dimensional-list-to-json-file
from _ctypes import PyObj_FromPtr  # see https://stackoverflow.com/a/15012814/355230
import json
import re


class NoIndent(object):
    """ Value wrapper. """
    def __init__(self, value):
        if not isinstance(value, (list, tuple)):
            raise TypeError('Only lists and tuples can be wrapped')
        self.value = value


class MyEncoder(json.JSONEncoder):
    FORMAT_SPEC = '@@{}@@'  # Unique string pattern of NoIndent object ids.
    regex = re.compile(FORMAT_SPEC.format(r'(\d+)'))  # compile(r'@@(\d+)@@')

    def __init__(self, **kwargs):
        # Keyword arguments to ignore when encoding NoIndent wrapped values.
        ignore = {'cls', 'indent'}

        # Save copy of any keyword argument values needed for use here.
        self._kwargs = {k: v for k, v in kwargs.items() if k not in ignore}
        super(MyEncoder, self).__init__(**kwargs)

    def default(self, obj):
        return (self.FORMAT_SPEC.format(id(obj)) if isinstance(obj, NoIndent)
                    else super(MyEncoder, self).default(obj))

    def iterencode(self, obj, **kwargs):
        format_spec = self.FORMAT_SPEC  # Local var to expedite access.

        # Replace any marked-up NoIndent wrapped values in the JSON repr
        # with the json.dumps() of the corresponding wrapped Python object.
        for encoded in super(MyEncoder, self).iterencode(obj, **kwargs):
            match = self.regex.search(encoded)
            if match:
                id = int(match.group(1))
                no_indent = PyObj_FromPtr(id)
                json_repr = json.dumps(no_indent.value, **self._kwargs)
                # Replace the matched id string with json formatted representation
                # of the corresponding Python object.
                encoded = encoded.replace(
                            '"{}"'.format(format_spec.format(id)), json_repr)

            yield encoded

In [1]:
import random
import numpy as np
import pandas as pd
from pathlib import Path
import json

PROJECT_PATH = Path("D:/Knee_Flexion_auditory_tactile_feedback/Analysis/results")
angles = [1, 2, 3, 4]
number_of_angles = len(angles)

n = {'Pre_test': 6,
      'RT-1' : 6,
      'Feedback-11' : 8,
      'Feedback-12' : 8,
      'Feedback-13' : 8,
      'RT-2' : 6,
      'Feedback-21' : 8,
      'Feedback-22' : 8,
      'Feedback-23' : 8,
      'RT-3' : 6,
      'Feedback-31' : 8,
      'Feedback-32' : 8,
      'Feedback-33' : 8,
      'RT-Final' : 6,
      'RT-15 min' : 6,
      'RT-24 hours': 6,
      'Transfer': 6}

conf = pd.read_csv(PROJECT_PATH / '_conf.csv')
conf = conf[conf.process==1]
conf

,participant,age,height,mass,sex,leg,feedback_group,conf_file,process
0,P0,30,167,61,1,L,T,D:\Knee_Flexion_auditory_tactile_feedback\Anal...,1
1,N0,30,167,61,1,L,T,D:\Knee_Flexion_auditory_tactile_feedback\Anal...,1
2,N1,30,167,61,1,L,T,D:\Knee_Flexion_auditory_tactile_feedback\Anal...,1


In [9]:
for iparticipant in conf.participant:
    number_of_angles = len(angles)
    number_tests = np.sum(list(n.values())) + len(n.keys())
    groups = np.zeros((number_of_angles, int(number_tests)))
    irow = 0
    header_row_indices = np.zeros(len(n.keys())).astype(int)
    groups.T[0] = np.ones(4)*10
    for i, igroup in enumerate(list(n.keys())):
        
        if i+1!=len(n.keys()):
            irow += n[igroup] + 1
            header_row_indices[i+1] = irow
            groups.T[irow] = np.ones(4)*10
        else:
            irow = groups.shape[1]
        
        number_of_angles = len(angles)
        number_participants = number_of_angles*n[igroup]
        participants = np.arange(1, number_participants+1, 1).tolist()

        i_group = 0

        while number_participants > 0 and number_of_angles > 1:
            team = random.sample(participants, n[igroup])
            groups[i_group, header_row_indices[i]+1 : irow] = np.sort(team)
            for x in team:
                participants.remove(x)
                number_participants -= 1
            number_of_angles -= 1
            i_group += 1
        groups[-1, header_row_indices[i]+1 : irow] = participants

    groups = groups.astype(int).T
    output_table = pd.DataFrame(groups)
    for i, irow in enumerate(header_row_indices):
        output_table.loc[[irow]]=[list(n.keys())[i], '','','']
    output_table.columns = angles
    
    output_path = PROJECT_PATH / iparticipant / 'angle_order.csv'
    output_table.to_csv(output_path, index=False)
    
    important_rows = np.hstack((header_row_indices,output_table.shape[0]))
    block_dict = {}
    for i, irow in enumerate(important_rows[:-1]):
        df_block = output_table.loc[irow+1:important_rows[i+1]-1] 
        angles_list = [int(np.argwhere(df_block.values == ivalue)[0][1]+1) for ivalue in np.sort(df_block.values.flatten())]
        angles_list = list(''.join(str(kk) + 'R' * (n % 32 == 31) for n, kk in enumerate(angles_list)))
        # angles_list = list(''.join(kk + '\\n' * (n % 10 == 9) for n, kk in enumerate(angles_list)))
        angles_list = ', '.join(list(map(lambda x: x.replace('R', '1min Rest'), angles_list)))
        block_dict[output_table.loc[irow].values[0]] = angles_list


    with open(PROJECT_PATH / iparticipant / 'angle_order.json', 'w') as fp:
        json.dump(block_dict, fp, cls=MyEncoder, indent=0)
        # json.dump(block_dict, fp, indent )
        # fp.write(',\n')